In [ ]:
%load_ext autoreload
%load_ext aymurai.devtools.magic
%autoreload 2

In [ ]:
import flair
import logging

flair.logger.setLevel(logging.ERROR)

In [ ]:
from aymurai.spacy.display import DocRender
from aymurai.datasets.ar_juz_pcyf_10.docs import ArgentinaJuzgadoPCyF10DocsDataset
from aymurai.datasets.ar_juz_pcyf_10.annotations import (
    ArgentinaJuzgadoPCyF10LabelStudioAnnotations,
)

render = DocRender()
docs = ArgentinaJuzgadoPCyF10LabelStudioAnnotations('/resources/pipe/label-studio/resos-annotations/30-nov/no-decision/').data
# docs = ArgentinaJuzgadoPCyF10DocsDataset().data
sample = docs[:10]

print(len(docs))

# full pipeline

In [ ]:
from aymurai.text.extraction import FulltextExtract
from aymurai.pipeline import AymurAIPipeline
from aymurai.models.flair.core import FlairModel
from aymurai.models.flair.utils import FlairTextNormalize, SubcatMatcher


config = {
    "preprocess": [
        # (
        #     FulltextExtract,
        #     {
        #         "errors": "ignore",
        #         "use_cache": True,
        #     },
        # ),
        (FlairTextNormalize, {}),
    ],
    "models": [
        (
            FlairModel,
            {
                "basepath": "/resources/pipelines/examples/flair-simple/FlairModel/model.pt",
                "split_doc": True,
                'device': 'cuda:0'
            },
        )
    ],
    "postprocess": [
        (
            SubcatMatcher, {}
        )
    ],
    "multiprocessing": {},
    # "use_cache": True,
    "use_cache": True,
    # 'log_level': 'debug'
}

pipeline = AymurAIPipeline(config)

In [ ]:
results = pipeline.preprocess(sample)
results = pipeline.predict(results)

In [ ]:
render(results[0])

# splitting doc pipeline

In [ ]:
from aymurai.text.extraction import FulltextExtract
from aymurai.pipeline import AymurAIPipeline
from aymurai.models.flair.core import FlairModel
from aymurai.models.flair.utils import FlairTextNormalize

config = {
    "preprocess": [
        (
            FulltextExtract,
            {
                "errors": "ignore",
                "use_cache": True,
            },
        ),
        (FlairTextNormalize, {}),
    ],
    "models": [
        (
            FlairModel,
            {
                "basepath": "/resources/pipelines/examples/flair-simple/FlairModel/model.pt",
                "split_doc": False,
                'device': 'cpu'
            },
        )
    ],
    "postprocess": [],
    "multiprocessing": {},
    # "use_cache": True,
    "use_cache": True,
    # 'log_level': 'debug'
}

pipeline = AymurAIPipeline(config)

In [ ]:
preprocessed = pipeline.preprocess([docs[0]])

In [ ]:
splitted_doc = pipeline.models[0].split_sentences(preprocessed[0]['data']['doc.text'])


In [ ]:
items = [
    {'path': 'dummy', 'data': {'doc.text': sentence.text}} for sentence in splitted_doc
]


In [ ]:
results = pipeline.predict(items)

In [ ]:
for result in results:
    render(result)

# pipeline i/o

In [ ]:
pipeline.save('/resources/pipelines/tests/v1')

In [ ]:
pipeline = pipeline.load('/resources/pipelines/tests/v1/')

In [ ]:
pipeline.config

In [ ]:
results = pipeline.predict(items)

In [ ]:
for result in results:
    render(result)